In [12]:
import pandas as pd
import numpy as np
import os
import pickle
from keras.optimizers import SGD, Adam, Nadam, RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential,Model,load_model
from keras.layers import Embedding,Conv1D,MaxPooling1D
from keras.layers.core import Dense, Activation,Dropout ,Flatten
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint,TensorBoard, ReduceLROnPlateau,EarlyStopping
from keras.applications import Xception
from keras import regularizers
from keras import backend as K
import keras
import numpy as np
import pandas as pd
import cv2
import os
import glob
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [13]:
seed = 120
np.random.seed(seed)

In [14]:
train_path = 'data_train.csv'
train_df = pd.read_csv(train_path)# Loading a csv file with headers 
train_df.head()

,sentence,x,y,w,l,area_percent,title,date,phone_num,address,index,content,total,brand_name,thank_you
0,LIKE CAFE,0.727273,0.031250,0.363636,0.052083,0.018939,0,0,0,0,0,0,0,1,0
1,"Số 10 Khúc Hạo, Ba Đình, Hà Nội",0.515152,0.093750,0.575758,0.044643,0.025703,0,0,0,1,0,0,0,0,0
2,4629719,0.878788,0.142857,0.212121,0.032738,0.006944,0,0,1,0,0,0,0,0,0
3,HOA ĐƠN THANH TOÁN.,0.272727,0.227679,0.575758,0.035714,0.020563,1,0,0,0,0,0,0,0,0
4,19/01/2014 8:49:59CH,0.060606,0.327381,0.727273,0.034226,0.024892,0,1,0,0,0,0,0,0,0


In [15]:
train_df = shuffle(train_df)
train_df.head()

,sentence,x,y,w,l,area_percent,title,date,phone_num,address,index,content,total,brand_name,thank_you
419,Ngày: 14/012018 Giờ: 17:25:30,0.121212,0.354626,0.969697,0.031938,0.030970,0,1,0,0,0,0,0,0,0
209,Khang - Cều Giây - HN,0.375000,0.113757,0.791667,0.025573,0.020245,0,0,0,1,0,0,0,0,0
553,Service Charge : RM 0.00,0.076923,0.640148,1.038462,0.024977,0.025938,0,0,0,0,0,0,1,0,0
337,PHIẾU THANH TOÁN,0.178571,0.566360,0.857143,0.051248,0.043927,1,0,0,0,0,0,0,0,0
152,Giảm 10%: 10.000,0.437500,0.896893,1.375000,0.036723,0.050494,0,0,0,0,0,0,1,0,0


In [16]:
X_train = train_df["sentence"].fillna("fillna").values
Y_train = train_df[["title", "date", "phone_num","address","index","content","total","brand_name","thank_you"]].values

In [17]:
print("Shape of X_train:",X_train.shape)
print("Shape of Y_train:",Y_train.shape)

Shape of X_train: (572,)
Shape of Y_train: (572, 9)


In [18]:
Tokenizer = Tokenizer()
print(X_train[0])

Ngày: 14/012018 Giờ: 17:25:30


In [19]:
print("(Input->Sentence) Length of X_train:",X_train.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels
texts = X_train
print(texts[1])

(Input->Sentence) Length of X_train: (572,)
(output->Labels) Length of Y_train: (572, 9)
Khang - Cều Giây - HN


In [20]:
Tokenizer.fit_on_texts(texts) 
Tokenizer_vocab_size = len(Tokenizer.word_index) + 1
print("Tokenizer vocabulary size:",Tokenizer_vocab_size)

Tokenizer vocabulary size: 1004


In [21]:
len(max(X_train,key=len))

43

In [22]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [23]:
print("(Input->Sentence) Length of X_train:",X_train.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels

(Input->Sentence) Length of X_train: (457,)
(output->Labels) Length of Y_train: (457, 9)


In [24]:
X_train_encoded_words = Tokenizer.texts_to_sequences(X_train)
X_val_encoded_words = Tokenizer.texts_to_sequences(X_val)

In [25]:
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labelsprint("(Input->Sentence)

(output->Labels) Length of Y_train: (457, 9)


In [26]:
maxWordCount= 5500
maxDictionary_size=Tokenizer_vocab_size
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen=maxWordCount)
X_val_encoded_padded_words = sequence.pad_sequences(X_val_encoded_words, maxlen=maxWordCount)

In [27]:
print("(Input->Sentence) Length of X_train:",X_train_encoded_padded_words.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels

(Input->Sentence) Length of X_train: (457, 5500)
(output->Labels) Length of Y_train: (457, 9)


In [28]:
print(Y_train.shape)
print(Y_val.shape)

(457, 9)
(115, 9)


In [29]:
# Model

model = Sequential()

model.add(Embedding(maxDictionary_size, 32, input_length=maxWordCount)) #to change words to ints
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))
# model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
 #hidden layers
model.add(LSTM(10))
# model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(1200, activation='relu',W_constraint=maxnorm(1)))
# model.add(Dropout(0.6))
model.add(Dense(500, activation='relu',W_constraint=maxnorm(1)))

# model.add(Dropout(0.5))
 #output layer
model.add(Dense(9, activation='softmax'))

# Compile model
# adam=Adam(lr=learning_rate, beta_1=0.7, beta_2=0.999, epsilon=1e-08, decay=0.0000001)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5500, 32)          32128     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                1720      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1200)              13200     
_________________________________________________________________
dense_2 (Dense)              (None, 500)               600500    
_________________________________________________________________
dense_3 (Dense)      

/home/hung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1200, activation="relu", kernel_constraint=<keras.con...)`
  from ipykernel import kernelapp as app
/home/hung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(500, activation="relu", kernel_constraint=<keras.con...)`


In [30]:
learning_rate=0.0001
epochs = 25
batch_size = 32 #32
sgd = SGD(lr=learning_rate, nesterov=True, momentum=0.7, decay=1e-4)
Nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=Nadam, metrics=['accuracy'])

In [31]:
print(X_train_encoded_padded_words.shape)
print(Y_train.shape)

print(X_val_encoded_padded_words.shape)
print(Y_val.shape)

(457, 5500)
(457, 9)
(115, 5500)
(115, 9)


In [32]:
history  = model.fit(X_train_encoded_padded_words,Y_train, epochs = epochs, batch_size=batch_size, verbose=1,
validation_data=(X_val_encoded_padded_words, Y_val))

Instructions for updating:
Use tf.cast instead.
Train on 457 samples, validate on 115 samples
Epoch 1/25
457/457 [==============================] - 23s 51ms/step - loss: 1.9345 - acc: 0.3414 - val_loss: 1.9486 - val_acc: 0.2087
Epoch 2/25
457/457 [==============================] - 23s 50ms/step - loss: 1.6800 - acc: 0.3501 - val_loss: 1.5822 - val_acc: 0.3913
Epoch 3/25
457/457 [==============================] - 22s 49ms/step - loss: 1.4203 - acc: 0.4486 - val_loss: 1.5211 - val_acc: 0.4087
Epoch 4/25
457/457 [==============================] - 22s 49ms/step - loss: 1.2203 - acc: 0.5120 - val_loss: 1.3875 - val_acc: 0.5130
Epoch 5/25
457/457 [==============================] - 22s 49ms/step - loss: 1.1095 - acc: 0.5624 - val_loss: 1.3625 - val_acc: 0.4696
Epoch 6/25
457/457 [==============================] - 22s 49ms/step - loss: 0.9463 - acc: 0.6477 - val_loss: 1.2816 - val_acc: 0.6087
Epoch 7/25
457/457 [==============================] - 23s 50ms/step - loss: 0.8802 - acc: 0.6477 - val

In [33]:
score = model.evaluate(X_val_encoded_padded_words, Y_val, verbose=1)
print('Test accuracy:', score[1],'%')

115/115 [==============================] - 1s 8ms/step
Test accuracy: 0.6260869611864505 %


In [34]:
phrase = "Tên menu Đơn giá SL Tiền"
tokens = Tokenizer.texts_to_sequences([phrase])
tokens = pad_sequences(tokens, maxlen=5500)
prediction = model.predict(np.array(tokens))
i,j = np.where(prediction == prediction.max()) #calculates the index of the maximum element of the array across all axis
# i->rows, j->columns
i = int(i)
j = int(j)
print(prediction)
total_possible_outcomes = ["area_percent", "title", "date", "phone_num","address","index","content","total","brand_name","thank_you"]
print("Result:",total_possible_outcomes[j])

[[4.6446081e-03 2.1676740e-02 1.1444956e-01 1.1101150e-02 3.9070779e-01
  1.2874262e-03 1.1548631e-04 2.4770917e-02 4.3124634e-01]]
Result: brand_name


In [35]:
# serialize model to JSON
model_json = model.to_json()
with open("model_sequence_only.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_sequence_only.h5")
print("Saved model to disk")

with open('tokenizer_sequence_only.pickle', 'wb') as handle:
    pickle.dump(Tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Saved model to disk
